In [ ]:
# Step 1: Install necessary libraries
!pip install pandas boto3 datasets evaluate transformers[sentencepiece] accelerate seqeval mlflow optimum[openvino,nncf] -q

## Dataset manipulation & env preparation

In [ ]:
### Load all pipeline parameters
import os
key_id = os.environ.get('S3_ACCESS_KEY_ID')
secret_key = os.environ.get('S3_SECRET_ACCESS_KEY')
model_name= os.environ.get("model_name")
bucket_name= os.environ.get("bucket")
version_model=os.environ.get("version")
run_name=os.environ.get("run_name")

In [ ]:
#Define your input column and your output column

input_column_name="text"
output_column_name="sentiment"

In [ ]:
import pandas as pd

labeled_dataset = "dataset.csv"

# Assuming the file is in the current working directory
df = pd.read_csv(labeled_dataset)

# Display the first few rows of the dataframe
df.head()

## Give a name to your model and version  🧙‍♂️🧙‍♂️

This process is crucial mainly because a `text-classification` model can be intended for a huge amount of approaches

In [ ]:
print(f"Your model is going to be saved with name `{model_name}` & version `{version_model}`")
print(f"\n Under directory models/openvino/text-classification/{model_name}/{version_model}")
model_path_s3=f"models/openvino/text-classification/{model_name}/{version_model}"

### Model manipulation 

In [ ]:
import json

# Opening JSON file
file_label2id = open('label2id.json')
file_id2label = open('id2label.json')
label2id = json.load(file_label2id)
id2label=json.load(file_id2label)
print(f"The label2id json loaded correctly: {label2id}")
print(f"The id2label json loaded correctly: {id2label}")
os.remove("label2id.json")
os.remove("id2label.json")

In [ ]:
# Add the new 'label' column to the dataframe by mapping values from the 'category' column
df['label'] = df[output_column_name].replace(label2id)
df.head(3)

In [ ]:
test_size = 0.2 ## Define the testing size for metrics

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=test_size)

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
from transformers import AutoTokenizer

base_model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples[input_column_name], truncation=True)

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=len(label2id),id2label=id2label, label2id=label2id)

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
#Mlflow
import mlflow

os.environ["MFLOW_EXPERIMENT_NAME"]=model_name

In [ ]:
training_args = TrainingArguments(
    hub_model_id=model_name,
    output_dir=run_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    eval_dataset=tokenized_test
)

In [ ]:
import mlflow

mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))
mlflow.set_experiment(model_name)

In [ ]:
### Fine tune model
trainer.train()

In [ ]:
mlflow.end_run()

In [ ]:
## Save pytorch 
trainer.save_model(model_name)

In [ ]:
!pip install optimum[openvino,nncf] -q

In [ ]:
from optimum.intel import OVModelForSequenceClassification

ov_model= OVModelForSequenceClassification.from_pretrained(model_name,export=True)
ov_model.save_pretrained(f"{model_name}_openvino")

In [ ]:
import boto3
from datetime import datetime


session = boto3.session.Session()
s3_client = session.client('s3',
                        region_name='nyc3',
                        endpoint_url='https://nyc3.digitaloceanspaces.com',
                        aws_access_key_id=key_id,
                        aws_secret_access_key=secret_key)

def uploadModel(path):
     for root,dirs,files in os.walk(path):
            for file in files:
                print(f"Uploading file: {file}")
                s3_client.upload_file(os.path.join(root,file),bucket_name,f"models/openvino/text-classification/{model_name}/{version_model}/{file}")

In [ ]:
#Upload ov binary to S3 & mlflow run
uploadModel(f"{model_name}_openvino")